In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('PS_20174392719_1491204439457_log.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
df.drop(['isFlaggedFraud','nameOrig','nameDest'],axis=1,inplace=True)

In [4]:
df.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


In [5]:
df['type'].value_counts()

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64

In [6]:
df1=df[df['type']=='CASH_OUT']
df2=df[df['type']=='PAYMENT']
df3=df[df['type']=='CASH_IN']
df4=df[df['type']=='TRANSFER']
df5=df[df['type']=='DEBIT']

In [7]:
df1['isFraud'].value_counts()

0    2233384
1       4116
Name: isFraud, dtype: int64

In [8]:
df2['isFraud'].value_counts()

0    2151495
Name: isFraud, dtype: int64

In [9]:
df3['isFraud'].value_counts()

0    1399284
Name: isFraud, dtype: int64

In [10]:
df4['isFraud'].value_counts()

0    528812
1      4097
Name: isFraud, dtype: int64

In [11]:
df5['isFraud'].value_counts()

0    41432
Name: isFraud, dtype: int64

In [12]:
df1=df[(df.type=='CASH_OUT') | (df.type=='TRANSFER')]

In [13]:
df1.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
2,1,TRANSFER,181.00,181.0,0.0,0.0,0.00,1
3,1,CASH_OUT,181.00,181.0,0.0,21182.0,0.00,1
15,1,CASH_OUT,229133.94,15325.0,0.0,5083.0,51513.44,0
19,1,TRANSFER,215310.30,705.0,0.0,22425.0,0.00,0
24,1,TRANSFER,311685.89,10835.0,0.0,6267.0,2719172.89,0


In [14]:
Type=pd.get_dummies(df1['type'],drop_first=True)

In [15]:
df2=pd.concat([df1,Type],axis=1)

In [16]:
df2.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,TRANSFER
2,1,TRANSFER,181.00,181.0,0.0,0.0,0.00,1,1
3,1,CASH_OUT,181.00,181.0,0.0,21182.0,0.00,1,0
15,1,CASH_OUT,229133.94,15325.0,0.0,5083.0,51513.44,0,0
19,1,TRANSFER,215310.30,705.0,0.0,22425.0,0.00,0,1
24,1,TRANSFER,311685.89,10835.0,0.0,6267.0,2719172.89,0,1


In [18]:
df2.drop('type',axis=1,inplace=True)

In [19]:
df2.drop('step',axis=1,inplace=True)

In [20]:
df2.shape

(2770409, 7)

In [21]:
from sklearn.preprocessing import RobustScaler

In [22]:
rs=RobustScaler()
d=rs.fit_transform(df2)
data=pd.DataFrame(d)
data.columns=df2.columns
data.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,TRANSFER
0,-0.764372,-0.003967,0.0,-0.345628,-0.461335,1.0,1.0
1,-0.764372,-0.003967,0.0,-0.332454,-0.461335,1.0,0.0
2,0.258572,0.484597,0.0,-0.342467,-0.432635,0.0,0.0
3,0.196809,0.012938,0.0,-0.331681,-0.461335,0.0,1.0
4,0.627408,0.339744,0.0,-0.341730,1.053601,0.0,1.0


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train,x_test,y_train,y_test=train_test_split(data.drop('isFraud',axis=1),data['isFraud'],test_size=0.3,random_state=42)

In [25]:
from sklearn import tree

In [26]:
dt=tree.DecisionTreeClassifier()
dt.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [27]:
dt.score(x_test,y_test)

0.9993370415690578

In [28]:
y_pred=dt.predict(x_test)

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[828391,    236],
       [   315,   2181]], dtype=int64)

In [30]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=30)
model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [31]:
model.score(x_test,y_test)

0.9992516149835825

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(n_estimators=100,random_state=0,)
gb.fit(x_train,y_train)
y_pred=gb.predict(x_test)


In [34]:
gb.score(x_test,y_test)

0.9975827885884521

In [35]:
x_train,x_test,y_train,y_test=train_test_split(data.drop('isFraud',axis=1),data['isFraud'],test_size=0.3,random_state=42)

In [36]:
from sklearn.ensemble import AdaBoostClassifier
ab=AdaBoostClassifier(n_estimators=150,random_state=0)
ab.fit(x_train,y_train)
y_pred=ab.predict(x_test)
ab.score(x_test,y_test)

0.9986680671813919